In [1]:
import tensorflow as tf

import tensorflow_datasets as tfds

celeba_bldr = tfds.builder('celeb_a')
celeba_bldr.download_and_prepare()
celeba = celeba_bldr.as_dataset(shuffle_files=True)
celeba_train = celeba['train']
celeba_valid = celeba['validation']
celeba_test = celeba['test']

def count_items(ds):
    n = 0
    for _ in ds:
        n += 1
    return n
    
print('Train set: {}'.format(count_items(celeba_train)))
print('Validation set: {}'.format(count_items(celeba_valid)))
print('Test set: {}'.format(count_items(celeba_test)))

2023-02-23 13:11:43.913968: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 13:11:47.637958: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 13:11:47.637987: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-23 13:11:58.059355: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

Dl Completed...: 100%|██████████| 4/4 [00:00<00:00, 46.95 url/s]


KeyboardInterrupt: 

: 

### 画像変換とデータ拡張

***

In [ ]:
import matplotlib.pyplot as plt
exmaples = []
for example in celeba_train.take(5):
    exmaples.append(example['image'])
fig = plt.figure(figsize=(16, 8.5))
# 列１:画像を短形に切り抜く
ax =  fig.add_subplot(2, 5, 1)
ax.set_title('Crop to a /nbouding_box', size = 15)
ax.imshow(exmaples[0])
ax = fig.add_subplot(2, 5, 6)
img_cropped = tf.image.crop_to_bounding_box(exmaples[0], 50, 20, 128, 128)
ax.imshow(img_cropped)
# 列２:画像を水平方向に反転 
ax = fig.add_subplot(2, 5, 2)
ax.set_title('Flip (horizontal)', size = 15)
ax.imshow(exmaples[1])
ax = fig.add_subplot(2, 5, 7)
img_flipped = tf.image.flip_left_right(exmaples[1])
ax.imshow(img_flipped)
# 列３:コントラストを調整
ax = fig.add_subplot(2, 5, 3)
ax.set_title('Adjust contrast', size = 15)
ax.imshow(exmaples[2])
ax = fig.add_subplot(2, 5, 8)
img_const = tf.image.adjust_contrast(exmaples[2], 2)
ax.imshow(img_const)
# 列４:明度を調整する
ax = fig.add_subplot(2, 5, 4)
ax.set_title('Adjust brightness', size = 15)
ax.imshow(exmaples[3])
ax = fig.add_subplot(2, 5, 9)
img_bright = tf.image.adjust_brightness(exmaples[3], 0.4)
ax.imshow(img_bright)
# 列５:画像を中心で切り抜き、元のサイズ(218x178)にリサイズする
ax = fig.add_subplot(2, 5, 5)
ax.set_title('Center crop', size = 15)
ax.imshow(exmaples[4])
ax = fig.add_subplot(2, 5, 10)
img_center_crop = tf.image.central_crop(exmaples[4], 0.7)
img_resized = tf.image.resize(img_center_crop, size = (218,178))
ax.imshow(img_resized)
plt.show()

In [ ]:
tf.random.set_seed(1)
fig = plt.figure(figsize=(14, 12))
for i,example in enumerate(celeba_train.take(3)):
    image = example['image']
    ax = fig.add_subplot(3, 2, 2*i+1)
    ax.imshow(image)
    if i == 0:
        ax.set_title('Original image', size=15)
    ax = fig.add_subplot(3, 4, 1*4+2)
    img_crop = tf.image.random_crop(image, size = (178, 178, 3))
    ax.imshow(img_crop)
    if i == 0:
        ax.set_title('Random crop', size=15)
    ax = fig.add_subplot(3, 4, 2*4+2)
    img_flip = tf.image.random_flip_left_right(image)
    ax.imshow(img_flip)
    if i == 0:
        ax.set_title('Random flip', size=15)
    ax = fig.add_subplot(3, 4, 3*4+2)
    img_contrast = tf.image.random_contrast(image, 0.5, 1.5)
    ax.imshow(img_contrast)
    if i == 0:
        ax.set_title('Random contrast', size=15)
    ax = fig.add_subplot(3, 4, 4*4+2)
    img_brightness = tf.image.random_brightness(image, 0.5)
    ax.imshow(img_brightness)
    if i == 0:
        ax.set_title('Random brightness', size=15)
plt.show()

### CNN性別分類器を訓練する

***

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding = 'same' , activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(rate = 0.5),

    tf.keras.layers.Conv2D(64, (3, 3), padding = 'same' , activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(rate = 0.5),

    tf.keras.layers.Conv2D(128, (3, 3), padding = 'same' , activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(256, (3, 3), padding = 'same' , activation='relu'),
])

